In [1]:
from nn import TimeSeriesTrainer, TimeSeriesNN
import torch
import torch.nn as nn
from dataset import TimeSeriesDataset
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np
from torchvision.transforms import Compose
from tqdm import tqdm
import matplotlib.pyplot as plt

In [8]:
# Constants
SEQ_LENGTH = 10
VECTOR_SIZE = SEQ_LENGTH*12
BATCH_SIZE = 1024
DEVICE = 'mps'

In [9]:
# Transform functions for dataset

class Normalise:
    
    def __init__(self,
                 train_data,
                 seq_length) -> None:
        self.train_data = train_data
        self.mean = torch.Tensor(np.array(self.train_data.mean())).repeat(seq_length)
        self.std = torch.Tensor(np.array(self.train_data.std())).repeat(seq_length)
           
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        if x.shape[-1]>1:
            x = (x-self.mean)/self.std
        elif x.shape[-1]==1:
            x = (x-self.mean[-1])/self.std[-1]
        return x
    
    
def to_tensor(x: np.ndarray):
    if x.ndim>0:
        return torch.Tensor(x).float()
    else:
        return torch.Tensor([x]).float()

In [10]:
train_data_df = pd.read_csv('train_delhi_aqi.csv')
val_data_df = pd.read_csv('val_delhi_aqi.csv')

transform = Compose([
    to_tensor,
    Normalise(train_data_df,SEQ_LENGTH)
])

train_data = TimeSeriesDataset(train_data_df,10,transform)
val_data = TimeSeriesDataset(val_data_df,10,transform)

train_loader = DataLoader(train_data,BATCH_SIZE,True)
val_loader = DataLoader(val_data,BATCH_SIZE)

In [11]:
# Linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim: int):
        super().__init__()
        self.linear = nn.Linear(input_dim,1)

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel(VECTOR_SIZE)

In [13]:
trainer = TimeSeriesTrainer(model,train_loader,val_loader,device=DEVICE)

In [14]:
trainer.train(2)

Epoch 1, Train Loss: 0.0842, Val Loss: 0.1064


Epoch 2, Train Loss: 0.0574, Val Loss: 0.1066
Finished Training


In [15]:
# Result on test data
test_data_df = pd.read_csv('test_delhi_aqi.csv')
test_data = TimeSeriesDataset(test_data_df,10,transform)
test_loader = DataLoader(test_data,BATCH_SIZE)

trainer.val_loader = test_loader

trainer.validate_epoch()

0.09693218581633384

In [16]:
def get_pred_vals(model,test_loader):
    pred_vals = []
    targets = []
    model.eval()  
    loop = tqdm(test_loader, desc="Validation", leave=False)
    with torch.no_grad():
        for inputs, target in loop:
            outputs = model(inputs)
            pred_vals.append(outputs)
            targets.append(target)
    pred_vals = torch.cat(pred_vals).view(-1).cpu().numpy()
    targets = torch.cat(targets).view(-1).cpu().numpy()
    return pred_vals,targets


def plot(pred_vals, targets):
    plt.figure(figsize=(12, 6))

    plt.plot(pred_vals, label='Predicted Values', color='blue', linestyle='-')
    
    plt.plot(targets, label='Actual Targets', color='red', linestyle='-')

    # Adding title and labels
    plt.title('Comparison of Predictions and Actual AQI(Normalised) Over Time')
    plt.xlabel('Time (Hrs)')
    plt.ylabel('AQI (Normalised)')
    plt.legend(loc='best')  

    plt.grid(True)

    plt.show()
    
def calculate_r2(pred_vals, targets):
    total_variance = np.sum((targets - np.mean(targets))**2)
    residual_variance = np.sum((targets - pred_vals)**2)
    # R^2 score
    r2_score = 1 - (residual_variance / total_variance)
    return r2_score

In [18]:
pred_vals,targets = get_pred_vals(model.to('cpu'),test_loader)

Validation:   0%|          | 0/137 [00:00<?, ?it/s]

In [19]:
plot(pred_vals[:1000],targets[:1000])

In [22]:
calculate_r2(pred_vals, targets)

0.7933948487043381